In [43]:
import pandas as pd
import glob
import os
import numpy  as np
import seaborn as sns        
import matplotlib.pyplot as plt    
from transformers import AutoTokenizer, AutoModel
import torch
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report



In [10]:

df=pd.read_csv('/Users/danielmunoz/Documents/EDUCACION/DATA_ANALIST/CURSOS/TFM/DATASET/170925dataset.csv')

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17976 entries, 0 to 17975
Data columns (total 33 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   video_id                17976 non-null  object 
 1   title                   17976 non-null  object 
 2   description             15128 non-null  object 
 3   published_at            17976 non-null  object 
 4   duration_minutes        17976 non-null  float64
 5   definition              17976 non-null  object 
 6   licensed_content        17976 non-null  bool   
 7   has_caption             17976 non-null  bool   
 8   category_id             17976 non-null  float64
 9   views                   17976 non-null  float64
 10  likes                   17976 non-null  float64
 11  comments                17976 non-null  float64
 12  tags                    11178 non-null  object 
 13  tags_count              17976 non-null  float64
 14  views_per_day           17976 non-null

In [33]:
df['channel_created'].tail(25)


17951           2016-01-20T15:58:48Z
17952           2014-05-18T13:29:54Z
17953           2013-04-07T22:25:25Z
17954    2024-02-05T13:40:27.140732Z
17955           2018-05-13T13:57:56Z
17956           2014-09-19T09:02:59Z
17957           2014-10-22T00:11:36Z
17958           2015-01-05T14:24:29Z
17959    2023-11-11T12:50:57.870349Z
17960           2019-08-04T09:10:56Z
17961           2005-12-16T09:01:28Z
17962           2013-01-31T13:19:49Z
17963           2008-05-23T18:22:20Z
17964    2021-01-18T17:21:55.597966Z
17965           2014-10-16T15:37:29Z
17966           2011-11-10T14:43:27Z
17967           2014-11-01T20:12:15Z
17968           2016-01-09T13:31:40Z
17969           2014-12-24T18:19:01Z
17970           2013-12-21T22:08:46Z
17971    2025-02-25T03:38:08.245051Z
17972           2015-03-11T15:42:33Z
17973           2014-11-09T13:15:54Z
17974           2014-05-15T23:57:55Z
17975           2016-11-23T02:14:51Z
Name: channel_created, dtype: object

In [31]:
df['channel_created'].tail(25)


17951           2016-01-20T15:58:48Z
17952           2014-05-18T13:29:54Z
17953           2013-04-07T22:25:25Z
17954    2024-02-05T13:40:27.140732Z
17955           2018-05-13T13:57:56Z
17956           2014-09-19T09:02:59Z
17957           2014-10-22T00:11:36Z
17958           2015-01-05T14:24:29Z
17959    2023-11-11T12:50:57.870349Z
17960           2019-08-04T09:10:56Z
17961           2005-12-16T09:01:28Z
17962           2013-01-31T13:19:49Z
17963           2008-05-23T18:22:20Z
17964    2021-01-18T17:21:55.597966Z
17965           2014-10-16T15:37:29Z
17966           2011-11-10T14:43:27Z
17967           2014-11-01T20:12:15Z
17968           2016-01-09T13:31:40Z
17969           2014-12-24T18:19:01Z
17970           2013-12-21T22:08:46Z
17971    2025-02-25T03:38:08.245051Z
17972           2015-03-11T15:42:33Z
17973           2014-11-09T13:15:54Z
17974           2014-05-15T23:57:55Z
17975           2016-11-23T02:14:51Z
Name: channel_created, dtype: object

In [34]:
df["text"] = (
    df["title"].fillna("") + " " +
    df["description"].fillna("") + " " +
    df["tags"].fillna("")
)

In [35]:
df["text"].head() 

0    ¿Cómo CURAR una GASTRITIS? | DrPoloGuerrero AN...
1    Lamine Yamal vs Como 1907 | 2 Goals (10/08/202...
2    Cómo Chile reinventó el Leopard 1v y así lo hi...
3    COMO se HACE el CONTRACHAPADO de MADERA |ASÍ s...
4    ¡SHOCK AÉREO GLOBAL! : ¿Cómo Rusia derriba un ...
Name: text, dtype: object

In [37]:
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

In [40]:
model_2 = SentenceTransformer("all-MiniLM-L6-v2")  # rápido y ligero
embeddings = model_2.encode(df["text"].tolist(), show_progress_bar=True)
emb_df = pd.DataFrame(embeddings, index=df.index)

Batches:   0%|          | 0/562 [00:00<?, ?it/s]

In [50]:
emb_df

,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
0,-0.025612,-0.008946,-0.068489,-0.008511,0.001675,-0.015000,0.085947,0.040911,0.036980,-0.059756,...,0.047394,-0.075907,0.032917,0.039514,0.082376,0.063752,-0.004195,0.069130,-0.026311,-0.020573
1,-0.096395,0.024196,-0.032604,-0.097147,0.033710,0.040789,0.055366,0.055789,0.100515,0.001278,...,0.026993,0.033986,-0.035761,-0.017136,0.048598,0.013904,0.079093,-0.044566,0.009570,-0.014261
2,-0.020472,-0.018498,0.001262,-0.069624,0.003939,-0.060070,-0.076335,0.027321,-0.066565,0.043797,...,0.013996,0.049441,0.112956,-0.055560,-0.034022,0.036058,0.022111,-0.045265,-0.025346,0.035816
3,-0.078822,-0.001135,-0.002196,-0.050082,0.021697,-0.022327,0.008090,-0.006924,-0.020526,0.056482,...,0.027743,-0.101173,0.026456,0.089399,0.051516,0.006606,0.059548,-0.018668,0.063205,0.002959
4,0.030191,0.005590,-0.030236,-0.046448,0.038494,-0.052229,0.031593,0.042906,-0.006005,0.028297,...,-0.008514,-0.001517,0.083356,-0.058123,0.013996,0.019939,0.028160,-0.007706,-0.078966,-0.000914
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17971,0.026301,-0.028241,-0.014450,-0.087323,0.000787,-0.031916,0.078817,0.010746,0.007404,0.005674,...,0.055893,0.005100,0.073682,-0.023375,0.069007,0.050659,-0.010090,0.037011,-0.068689,-0.038288
17972,-0.071031,0.060969,0.055865,-0.110255,0.003795,0.059403,0.031307,0.110903,-0.003893,0.018558,...,-0.020297,0.071269,-0.008098,0.039319,-0.005689,-0.004455,0.141406,0.049419,-0.025538,-0.017712
17973,-0.076197,0.091032,0.005058,-0.015634,0.029612,0.014104,0.059685,0.064875,0.020343,0.045068,...,0.053552,-0.026225,0.063448,0.007485,-0.017521,-0.027573,0.095531,0.019674,-0.090750,-0.001279
17974,-0.076149,-0.019762,0.030709,-0.113296,0.029909,0.010344,0.067584,0.033672,0.030676,0.014179,...,0.083563,0.047576,0.039789,0.041310,-0.010363,0.095272,0.003552,0.068349,-0.038438,-0.035973


In [58]:
X = emb_df

y = df["mostpopular"]  # target

In [59]:
X = X.copy()
X.columns = X.columns.astype(str)

# 2) Asegura que TODAS las features sean numéricas (RandomForest no traga 'object' / strings)
#    Si tienes categóricas, one-hot o conviértelas antes. Aquí fuerzo a numérico como ejemplo:
X = X.apply(pd.to_numeric, errors='coerce')

# 3) Trata los NaN (RandomForest no acepta NaN)
X = X.fillna(0)  # o un imputador mejor

# 4) Asegura que y sea 1D (Series) con dtype adecuado
y = pd.Series(y).astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y if y.nunique() > 1 else None
)

clf = RandomForestClassifier(n_estimators=200, random_state=42)
clf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=200, random_state=42)

In [60]:
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.97      0.94      2577
           1       0.92      0.75      0.83      1019

    accuracy                           0.91      3596
   macro avg       0.92      0.86      0.89      3596
weighted avg       0.91      0.91      0.91      3596

